## Import

In [316]:
import numpy as np
import os
import re
import nltk
from nltk import Tree
from copy import deepcopy
import pickle
import operator
from operator import itemgetter
import argparse
import networkx as nx

In [70]:
DIGITS = re.compile("[0-9]", re.UNICODE)

## Processing

In [23]:
# Importing the corpus
file_path = os.path.join(os.getcwd(), 'sequoia-corpus+fct.mrg_strict')

with open(file_path,'r') as f:
    file = f.read()
    sentences = file.split('\n')[0:]

In [24]:
# Split the dataset
corpus_length = len(sentences)

length_train = int(0.8 * corpus_length)
length_dev = int(0.1 * corpus_length)
length_test = int(0.1 * corpus_length)

end_dev = length_train + length_dev
 

corpus_train = sentences[:length_train]
corpus_dev = sentences[length_train:end_dev]
corpus_test = sentences[end_dev:]

## Utils

In [1]:
# Create file with non-parsed sentences
def unparse(sentence):
    sentence = sentence.split(' ')
    
    list_sentence = []
    
    for token in sentence:
        if ')' in token:
            word = ''
            for caract in token:
                if caract == ')':
                    break
                else:
                    word += caract
            list_sentence.append(word)
            
    sent = listToString(list_sentence)
    
    return sent 

In [25]:
# Function to convert list to string
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for idx, ele in enumerate(s):
        if idx == len(s)-1:
            str1 += ele
        else:
            str1 += ele + ' '  
    
    # return string   
    return str1 

In [26]:
# Remove functional tags
def functional_tags_out(sentence):
    sentence = sentence.split(' ')
    new_sentence = []
    for part_sent in sentence:
        if '(' in part_sent:
            new_part_sent = ''
            for caract in part_sent:
                if caract == '-':
                    break
                new_part_sent += caract
        else:
            new_part_sent = part_sent
            
        new_sentence.append(new_part_sent)
    
    return listToString(new_sentence)

In [27]:
# Add rule to dictionary
def add_dic(dico, tag, rules, count):
    if tag not in dico.keys():
        dico[tag] = {}
    if rules not in dico[tag]:
        dico[tag][rules] = count
    else:
        dico[tag][rules] += count

In [56]:
# Normalize counts
def count_to_freq(dico):
    for tag in dico:
            total = 0
            for rule in dico[tag].keys():
                total += dico[tag][rule]
            
            
            for rule in dico[tag].keys():
                dico[tag][rule] /= total

In [58]:
# Get all tags from grammar
def get_tags(dico):
    tags_list = []
    for tag in dico:
        if tag not in tags_list:
            tags_list.append(tag)
            
        for rule in dico[tag].keys():
            list_child_tags = rule.split(' ')
            for child_tag in list_child_tags:
                if child_tag not in tags_list:
                    tags_list.append(child_tag)
                    
    return tags_list                  

In [66]:
# In case the word is not available in the vocabulary,
# we can try multiple case normalizing procedure.
# We consider the best substitute to be the one with the lowest index,
# which is equivalent to the most frequent alternative.
def case_normalizer(word, dictionary):
    w = word
    lower = (dictionary.get(w.lower(), 1e12), w.lower())
    upper = (dictionary.get(w.upper(), 1e12), w.upper())
    title = (dictionary.get(w.title(), 1e12), w.title())
    results = [lower, upper, title]
    results.sort()
    index, w = results[0]
    if index != 1e12:
        return w
    return word

In [67]:
# If the word is OOV, find the closest alternative
def normalize(word, word_id):
    if word not in word_id:
        word = DIGITS.sub("#", word)
    
    if word not in word_id:
        word = case_normalizer(word, word_id)

    if word not in word_id:
        return None
    
    return word

In [83]:
# Sorts words according to their Euclidean distance.
# To use cosine distance, embeddings has to be normalized so that their l2 norm is 1.
# indeed (a-b)^2"= a^2 + b^2 - 2a^b = 2*(1-cos(a,b)) of a and b are norm 1"""
def l2_nearest(embeddings, query_embedding, k):
    distances = (((embeddings - query_embedding) ** 2).sum(axis=1) ** 0.5)
    sorted_distances = sorted(enumerate(distances), key=itemgetter(1))
    return zip(*sorted_distances[:k])

In [262]:
# Convert list to parsed sentence
def list_to_parsed_sentence(parsing):
        if type(parsing) == str:
            return parsing

        else:
            string = ""
            for p in parsing:
                root_tag = p[0]
                parsing_substring = p[1]
                string = string + "(" + root_tag + " " + list_to_parsed_sentence(parsing_substring) + ")" + " "
            string = string[:-1]  # Remove the extra space
            return string

In [308]:
# Returns a Tree from a tagged sentence
def tagged_sent_to_tree(tagged_sent, remove_after_hyphen=True):
    max_id_node = 0

    tree = nx.DiGraph()

    sent = tagged_sent.split()
    hierarchy = list()

    hierarchy.append([])

    level = 0  # difference between the number of opened and closed parenthesis

    for (idx_bloc, bloc) in enumerate(sent):

        if bloc[0] == "(":

            if remove_after_hyphen:
                tag = clean_tag(bloc[1:])  # we add it to the hierarchy
            else:
                tag = bloc[1:]
            if level < len(hierarchy):  # there is already one tag as its level
                hierarchy[level].append((tag, max_id_node))
            else:  # first tag as its level
                hierarchy.append([(tag, max_id_node)])
            if idx_bloc > 0:
                tree.add_node(max_id_node, name=tag)
                max_id_node += 1
            level += 1

        else:

            word = ""
            nb_closing_brackets = 0
            for caract in bloc:
                if caract == ")":
                    nb_closing_brackets += 1
                else:
                    word += caract

            tree.add_node(max_id_node, name=word)
            tree.add_edge(max_id_node - 1, max_id_node)
            max_id_node += 1

            level -= nb_closing_brackets

            for k in range(nb_closing_brackets - 1, 0, -1):
                root = hierarchy[-2][-1][0]  # root tag
                id_root = hierarchy[-2][-1][1]
                if root == '':
                    break
                tags = hierarchy[-1]  # child tags

                for tag in tags:
                    tree.add_edge(id_root, tag[1])

                hierarchy.pop()

    return tree

In [309]:
# Partial sentence from subtree rooted at node
def tree_to_sentence_helper(tree, node):
    children = list(tree.successors(node))
    if (len(children) == 1) and (len(list(tree.successors(children[0]))) == 0):
        return "(" + tree.nodes[node]["name"] + " " + tree.nodes[children[0]]["name"] + ")"
    else:
        res = "(" + tree.nodes[node]["name"]
        for child in sorted(children):
            res += " " + tree_to_sentence_helper(tree, child)
        res += ")"
        return res

In [310]:
# Tree to sentence"""
def tree_to_sentence(tree):
    root = list(nx.topological_sort(tree))[0]
    return "( " + tree_to_sentence_helper(tree, root) + ")"

## PCFG Tree

In [5]:
class Node:
    def __init__(self, parent, value):
        self.parent = parent
        self.value = value
        self.children = []

    def GetChildrenValues(self):
        return list(map(lambda x: x.value, self.children))

In [192]:
class PCFG_Tree:
    def __init__(self, sentence):
        self.root = Node(None, 'SENT')
        self.grammar = {'SENT' : {}}
        self.lexicon = {}
        self.inv_lexicon = {}
        
        sentence = functional_tags_out(sentence)
        current_node = self.root
        sentence = sentence.split(' ')
        sentence = sentence[2:]
        
        
        for idx, word in enumerate(sentence):
            if '(' in word:
                current_tag = word.replace('(', '')
                new_node = Node(current_node, current_tag)
                current_node.children.append(new_node)
                current_node = new_node
                
                if current_tag not in self.grammar:
                    self.grammar[current_tag] = {}
                    
            else:
                num_closed_brackets = 0
                ele = ''
                for caract in word:
                    if caract == ')':
                        num_closed_brackets += 1
                    else:
                        ele += caract
                        
                
                add_dic(self.inv_lexicon, ele, current_tag, 1)
                        
                add_dic(self.lexicon, current_tag, ele, 1)
                
                for i in range(num_closed_brackets):
                    if current_node.parent is not None:
                        current_node = current_node.parent
    
    def ExtractGrammar(self, node=None):
        if node is None:
            node = self.root
        if len(node.children) > 0:
            rule = listToString(node.GetChildrenValues())
            if rule not in self.grammar[node.value]:
                self.grammar[node.value][rule] = 1
            else:
                self.grammar[node.value][rule] += 1
            
            for child in node.children:
                self.ExtractGrammar(child)

## PCFG

In [229]:
# Class to build the PCFG in Chomsky normal form
class PCFG:

    def __init__(self, corpus):

        # Initialize grammar, lexicon and new tags
        self.grammar = {}
        self.lexicon = {}
        self.inv_lexicon = {}
        self.set_new_tags = set()

        # Build PCFG from corpus
        self.build_pcfg(corpus)

        # Build dictionnary of words form the corpus with the corresponding frequency 
        # {word : frequency}
        self.tokens = {}
        for tag in self.lexicon.keys():
            for word in self.lexicon[tag].keys():
                if word in self.tokens.keys():
                    self.tokens[word] += self.lexicon[tag][word]
                else:
                    self.tokens[word] = self.lexicon[tag][word]
        sum = np.sum(list(self.tokens.values()))
        
        for word in self.tokens:
            self.tokens[word] /= sum

            
        # Apply Chomsky normalization to PCFG from corpus
        self.to_chomsky_form()

        
        # Build dictionnary of terminal tags form the corpus with the corresponding frequency 
        # {terminal_tag : frequency}
        self.terminal_tags = {tag: np.sum(list(counts.values())) for (tag, counts) in self.lexicon.items()}
        sum = np.sum(list(self.terminal_tags.values()))
        for tag in self.terminal_tags:
            self.terminal_tags[tag] /= sum

        
        # Normalize
        count_to_freq(self.grammar)
        count_to_freq(self.lexicon)
        count_to_freq(self.inv_lexicon)

        
        # Get all tags
        list_all_tags = get_tags(self.grammar)
        
        # Tags lists
        self.list_new_tags = list(self.set_new_tags)
        self.list_tags = list(set(list_all_tags).difference(self.set_new_tags))
        self.list_all_tags = self.list_tags + self.list_new_tags
        self.dic_all_tags = {word: idx for (idx, word) in enumerate(self.list_all_tags)}
        
        self.nb_tags = len(self.list_tags)
        self.nb_all_tags = len(self.list_all_tags)

        
    # Apply Chomsky normalization to PCFG
    def to_chomsky_form(self):
        self.to_binary_rules()
        self.unit_rules_out()

        
    # Telescope unait rules
    def unit_rules_out(self):
        grammar_copy = deepcopy(self.grammar)
        lexicon_copy = deepcopy(self.lexicon)

        rules_to_remove = []

        for tag in grammar_copy.keys():
            
            for rule in grammar_copy[tag].keys():
                count = grammar_copy[tag][rule]
                list_tags = rule.split(' ')
                
                if len(list_tags) == 1: # Check if unit rule

                    child_tag = list_tags[0]
                    rules_to_remove.append((tag, child_tag))
                    proba = count / (np.sum(list(self.grammar[tag].values())))

                    # rule A -> B where B is a pre-terminal tag
                    if child_tag in lexicon_copy:
                        
                        if tag != "SENT":
                            new_tag = tag + "&" + child_tag
                            self.set_new_tags.add(new_tag)
                            
                            # new_tag -> word
                            for word in lexicon_copy[child_tag].keys():
                                count2 = lexicon_copy[child_tag][word]
                                add_dic(self.lexicon, new_tag, word, count2*proba)

                            
                            # for each rule X -> Y A, add rule X -> Y A&B
                            for tag2 in grammar_copy.keys():
                                for rule2 in grammar_copy[tag2].keys():
                                    list_tags = rule2.split(' ')
                                    count2 = grammar_copy[tag2][rule2]
                                    if len(list_tags) == 2 and list_tags[1] == tag:
                                        new_rule2 = rule2.replace(tag, new_tag)
                                        add_dic(self.grammar, tag2, new_rule2, count2)

                    # If B not pre-terminal, for each rule B -> X Y, add A -> X Y
                    else:
                    
                        for grand_child_tag in grammar_copy[child_tag].keys():
                            list_grand_child_tags = grand_child_tag.split(' ')
                            count3 = grammar_copy[child_tag][grand_child_tag]
                            if len(list_grand_child_tags) == 2:
                                add_dic(self.grammar, tag, grand_child_tag, count3*proba)
                

        for (left, right) in rules_to_remove:
            del self.grammar[left][right]
            
        for tag in grammar_copy.keys():
            if len(self.grammar[tag]) == 0:
                del self.grammar[tag]

    
    # Replace all rules with more than 2 children with a chain of rule
    def to_binary_rules(self):
        grammar_copy = deepcopy(self.grammar)
    
    
        for tag in grammar_copy.keys():
        
            for rule in grammar_copy[tag].keys():
            
                count = grammar_copy[tag][rule]
                tags_list = rule.split(' ')
            
                if len(tags_list) > 2: 
                    del self.grammar[tag][rule]
                    old_tag = tag
                    for idx, sub_tag in enumerate(tags_list[:-2]):
                        new_symbol = tag + '|' + '-'.join(tags_list[idx+1:])
                        self.set_new_tags.add(new_symbol)
                        new_rule = listToString([sub_tag, new_symbol])
                        add_dic(self.grammar, old_tag, new_rule, count)
                        old_tag = new_symbol
                    
                    last_two_tags = listToString(tags_list[-2:])
                    add_dic(self.grammar, old_tag, last_two_tags, count)
                                                     
 
                    
    # Build the PCFG
    def build_pcfg(self, corpus):
 
        for sentence in corpus:
            tree = PCFG_Tree(sentence)
            tree.ExtractGrammar()
        
            lexicon_sent = tree.lexicon
            inv_lexicon_sent = tree.inv_lexicon
            grammar_sent = tree.grammar
            
        
            # Add lexicon of sentence in general lexicon
            for tag in lexicon_sent.keys():
                for word in lexicon_sent[tag].keys():
                    add_dic(self.lexicon, tag, word, 1)
                    
            # Add inverse lexicon of sentence in general inverse lexicon
            for word in inv_lexicon_sent.keys():
                for tag in inv_lexicon_sent[word]:
                    add_dic(self.inv_lexicon, word, tag, 1)
                
            # Add grammar of sentence in general grammar      
            for tag in grammar_sent.keys():
                for rule in grammar_sent[tag].keys():
                    add_dic(self.grammar, tag, rule, 1)
                
            
        # Get rid of terminal tags
        self.grammar = { tag : dic for tag,dic in self.grammar.items() if len(dic) > 0}             

## OOV

In [320]:
# Class to handle OOV words
class OOV:
    
    def __init__(self, lexicon, list_all_tags, tokens):
        
        # Lexicon from PCFG
        self.lexicon = lexicon
        self.tokens = tokens
        
        # Words in the Polyglott embeddings
        self.poly_words, self.poly_embeddings = pickle.load(open('data/polyglot-fr.pkl', "rb"), 
                                                           encoding='bytes')
        
        # List of words in the lexicon                                    
        self.words_lexicon = list(tokens.keys())
        self.words_lexicon_id = {word: idx for (idx, word) in enumerate(self.words_lexicon)}
        
        # Assign Id's to each word in Polyglott embeddings
        self.poly_words_id = {word: idx for (idx, word) in enumerate(self.poly_words)}
        
        # Embedding matrix
        self.lexicon_embeddings = None
        
        # Words in the lexicon having a polyglott embedding
        self.lexicon_words_with_embed = []
        self.lexicon_words_with_embed_id = {}
        
        # Build embedding matrix with words from lexion
        self.build_embeddings_lexicon()
        self.lexicon_embeddings /= np.linalg.norm(self.lexicon_embeddings, axis=1)[:, None]  # normalize embeddings
        
        
        

        
    # Returns the closest word using a combination of levi distance and cosine similarity
    # of embeddings (if available)
    def closest_word(self, query):
            
        query_normalized = normalize(query, self.words_lexicon_id)
            
        # query_normalized exists in lexicon
        if query_normalized is not None:
            return query_normalized
            
        # Embedding of query exists in Polyglott embeddings, then return word
        # with closest embedding
        if query in self.poly_words:
            closest_word = self.closest_word_embedding(query)
            return closest_word
            
        # If embedding of query does not exists in in Polyglott embeddings, then return
        # word with closest levi distance and highest frequency
        else:
            closest_word = self.closest_word_levi(query,2)
            return closest_word
            
    
    # Build an emebdding matrix with words from the lexicon having on 
    # in the Polyglott embeddings
    def build_embeddings_lexicon(self):
        
        # Get embedding of the word if in the polyglott embeddings
        for word in self.words_lexicon:
            word_normalized = normalize(word, self.poly_words_id)
                
            if word_normalized is not None:
                self.lexicon_words_with_embed.append(word)
                id_word = self.poly_words_id[word_normalized]
                    
                if self.lexicon_embeddings is None:
                    self.lexicon_embeddings = self.poly_embeddings[id_word]
                else:
                    self.lexicon_embeddings = np.vstack([self.lexicon_embeddings, self.poly_embeddings[id_word]])

            
        # Assign new indexes to words having a polyglott embedding
        self.lexicon_words_with_embed_id = {w: i for (i, w) in enumerate(self.lexicon_words_with_embed)}

                
        
        
    # Returns word with closest embedding to query
    def closest_word_embedding(self, query):
            
            
        # Check if query has polyglott embedding
        query = normalize(query, self.poly_words_id)
            
        if query is None:
            print('Query has no embedding in the Polyglott embeddings')
            return None
            
            
        # If query has polyglott embedding, returns word in the lexicon
        # with closest embedding
        query_idx = self.poly_words_id[query]
        query_embedding = self.poly_embeddings[query_idx]
        query_embedding /= np.linalg.norm(query_embedding) # normalize
        indices, distances = l2_nearest(self.lexicon_embeddings, query_embedding, 1)
            
        idx = indices[0]
            
        closest_word = self.lexicon_words_with_embed[idx]
            
        return closest_word
        
    # Returns Levenshtein distance between two strings
    def levenshtein_distance(self, s1, s2):
        # Convert string to list
        list1 = list(s1)
        list2 = list(s2)
        N1 = len(list1)
        N2 = len(list2)
    
        # Initialize  Matrix
        mat = np.zeros((N1+1,N2+1))
        mat[:,0] = np.arange(N1+1)
        mat[0,:] = np.arange(N2+1)

        # Dynamic Programming
        for i in range(1, N1+1):
            for j in range(1, N2+1):
                if list1[i-1] == list2[j-1]:
                    mat[i, j] = min(mat[i-1, j]+1, mat[i,j-1]+1, mat[i-1,j-1])
                else:
                    mat[i, j] = min(mat[i-1,j]+1, mat[i, j-1]+1, mat[i-1,j-1]+1)
                
        return mat[-1,-1] 

        
        
    def closest_word_levi(self, query, k):
            
        candidates = {}
            
        # Separate candidates with different levi distance
        for i in range(1,k+1):
            candidates[i] = [] 
                
    
        min_dist = k
            
        # Find candidates
        for word in self.words_lexicon:
            levi_dist = self.levenshtein_distance(word, query)
            if levi_dist <= min_dist:
                candidates[levi_dist].append(word)
                min_dist = levi_dist
            
            
        # Find final candidates (with the lowest levi distance)
        final_candidates = None
        for i in range(1,k+1):
            if len(candidates[i]) > 0:
                final_candidates = candidates[i]
                break
            
        if final_candidates == None:
            print('No words within a levenshtein distance of {} for: {}'.format(k,query))
            return None
                    
                
        # Get frequency of each candidate
        freq_final_candidates = []
        for final_candidate in final_candidates:
            freq_final_candidates.append(self.tokens[final_candidate])
                
                
        idx_max_freq = np.argmax(freq_final_candidates)
            
        return final_candidates[idx_max_freq]         

## CYK

In [324]:
# Class to apply CYK algorithm
class CYK:
    
    def __init__(self, corpus):
        
        # PCFG and OOV class
        self.pcfg = PCFG(corpus)
        self.oov = OOV(self.pcfg.lexicon, self.pcfg.list_all_tags, self.pcfg.tokens)
        
        # Initialize CYP probability matrix
        self.proba_matrix = None
        self.cyk_matrix = None
              
        
    # Apply the CYK algorithm
    def CYK_algorithm(self, sentence):
        
        # Initialize
        n = len(sentence)
        r = self.pcfg.nb_all_tags
        P = np.zeros((n,n,r))
        cyk_matrix = np.zeros((n,n,r,3))
    
    
        # First level P[0,:,:]
        for idx_word, word in enumerate(sentence):
            
            # Get closest word in the lexicon
            word = self.oov.closest_word(word)
            
            if word is None:
                for idx_tag, tag in enumerate(self.pcfg.list_all_tags):
                    if tag in self.pcfg.terminal_tags:
                        P[0, idx_word, idx_tag] = self.pcfg.terminal_tags[tag]
                        
            else:
                for idx_tag, tag in enumerate(self.pcfg.list_all_tags):
                    if tag in self.pcfg.inv_lexicon[word]:
                        P[0, idx_word, idx_tag] = self.pcfg.inv_lexicon[word][tag]
                   
                        
                        
        # Other levels
        for l in range(1, n):
        
            for s in range(n-l):
            
                for tag in self.pcfg.grammar:
                    idx_tag = self.pcfg.dic_all_tags[tag]
                
                    for p in range(l):
                    
                        for rule in self.pcfg.grammar[tag]:
                            left_tag = rule.split(' ')[0]
                            right_tag = rule.split(' ')[1]
                            b = self.pcfg.dic_all_tags[left_tag]
                            c = self.pcfg.dic_all_tags[right_tag]
                    
                            prob_splitting = self.pcfg.grammar[tag][rule] * P[p, s, b] * P[l-p-1, s+p+1, c]
                        
                            if prob_splitting > P[l, s, idx_tag]:
                                P[l, s, idx_tag] = prob_splitting
                                cyk_matrix[l, s, idx_tag] = [p, b, c]
                        
                
                
                
        self.proba_matrix = P
        self.cyk_matrix = cyk_matrix.astype(int)
        
    # Remove new tags and de-telescope tags
    def clean_tags(self, tree):
        # remove new tags of type
        nodes = deepcopy(tree.nodes)
        for node in nodes:
            children = list(tree.successors(node))
            
            if len(children) == 0:
                pass
            
            elif len(children) == 1 and len(list(tree.successors(children[0]))) == 0:
                pass
            
            else:
                parent = list(tree.predecessors(node))
                if len(parent) == 0:
                    pass
                else:
                    tag = tree.nodes[node]["name"]
                    
                    if (self.pcfg.dic_all_tags[tag] >= self.pcfg.nb_tags) and (
                            "|" in tag):  
                        
                        for child in tree.successors(node):
                            tree.add_edge(parent[0], child)
                        tree.remove_node(node)

        # Decomposing A&B -> w into A -> B -> w
        max_node = np.max(tree.nodes())
        nodes = deepcopy(tree.nodes)
        for node in nodes:
            
            children = list(tree.successors(node))
            
            if len(children) == 0 or len(list(tree.predecessors(node))) == 0:
                pass
            
            elif len(children) == 1 and len(list(tree.successors(children[0]))) == 0:
                tag = tree.nodes[node]["name"]

                if (self.pcfg.dic_all_tags[tag] >= self.pcfg.nb_tags) and (
                        "&" in tag):  # new tag from unit rule
                    word = children[0]

                    idx_cut = None
                    
                    for (idx, c) in enumerate(tag):
                        if c == "&":
                            idx_cut = idx

                    tree.nodes[node]["name"] = tag[:idx_cut]

                    idx_pre_terminal_node = max_node + 1
                    tree.add_node(idx_pre_terminal_node, name=tag[idx_cut + 1:])
                    max_id_node += 1
                    tree.remove_edge(node, word)
                    tree.add_edge(node, idx_pre_terminal_node)
                    tree.add_edge(idx_pre_terminal_node, word)
    
    
    # Parse part of a sentence
    def parse_substring(self, s, l, idx_tag, sentence):
    

        if l == 0:
            return sentence[s]

        else: 
            cut = self.cyk_matrix[l, s, idx_tag, 0]
            idx_left_tag = self.cyk_matrix[l, s, idx_tag, 1]
            idx_right_tag = self.cyk_matrix[l, s, idx_tag, 2]

            left_tag = self.pcfg.list_all_tags[idx_left_tag]
            right_tag = self.pcfg.list_all_tags[idx_right_tag]

            return [[left_tag, self.parse_substring(s, cut, idx_left_tag, sentence)],
                    [right_tag, self.parse_substring(s + cut + 1, l - cut - 1, idx_right_tag, sentence)]]
    
    
    
    # Returns the parsed sentence
    def parse(self, sentence):
        
        sentence = sentence.split(' ')
        length_sentence = len(sentence)
        
        
        if length_sentence > 1:
            self.CYK_algorithm(sentence)
            idx_root_tag = self.pcfg.dic_all_tags['SENT']
            if self.proba_matrix[length_sentence - 1][0][idx_root_tag] == 0:  # no valid parsing
                return None
            parsing_list = self.parse_substring(0, length_sentence - 1, idx_root_tag, sentence)
            
        
        else:
            word = sentence[0]
            word_lexicon = self.oov.closest_word(word)
            
            
            if word_lexicon is None:
                tag = max(self.pcfg.terminal_tags, key=self.pcfg.terminal_tags.get)
                
            else:
                tag = max(self.pcfg.inv_lexicon[word_lexicon], key=self.pcfg.inv_lexicon[word_lexicon].get)
            
            parsing_list = '(' + tag + word + ')'
            
        # converting the parsing stored as a string into a tree
        tree = tagged_sent_to_tree("( (SENT " + list_to_parsed_sentence(parsing_list) + "))",
                                   remove_after_hyphen=False)
            
        self.clean_tags(tree)
            
            
        return tree_to_sentence(tree)
        

## Evaluation

In [ ]:
with open('eval_corpus.txt', 'r') as f:
    file = f.read()
    test_sentences = file.split('\n')

In [ ]:
# Build the parser with corpus_train
print('Building the parser...')
cyk_parser = CYK(corpus_train)
print('Done')

# Begin parsing
print('Parsing...')

with open('evaluation_data.parser.txt', 'w') as f:
    for sentence in test_sentences:
        print('Sentence: ' + sentence + '\n')
        parsed_sentence = cyk_parser.parse(sentence)
        if parsed_sentence is not None:
            print('Parsed sentence: ' + parsed_sentence + '\n')
            f.write('%s\n' % parsed_sentence)